# Imports

In [1]:
import os
os.chdir('..') # do not rerun this cell
print(os.getcwd())
from scripts import convert

import spacy
from tqdm.auto import tqdm

/home/jovyan/shared/2022_12_09_keyword_extraction


# Create Docs

In [2]:
convert.main('assets/macroeconomics_2e.jsonl', 100)

  0%|          | 0/94 [00:00<?, ?it/s]

# Get Docs

In [3]:
from spacyfuncs import spacyio

In [4]:
docs = spacyio.get_docs(None, 'corpus/sample.spacy')

Retrieving from existing Docbin file


  0%|          | 0/94 [00:00<?, ?it/s]

In [5]:
gold_terms = [doc.ents for doc in docs]

# Extract Keywords

In [6]:
nlp = spacy.load('training/model-best/')

In [7]:
doc_gen = tqdm(nlp.pipe([doc.text for doc in docs]), total=len(docs))
pred_terms = [doc.ents for doc in doc_gen]

  0%|          | 0/94 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (4364 > 4096). Running this sequence through the model will result in indexing errors
/home/jovyan/conda_envs/spacy/lib/python3.10/site-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [8]:
terms_for_eval = []

for gold, pred, doc in zip(gold_terms, pred_terms, docs):
    section_name = doc._.name.split('/')[-1]
    terms_for_eval.append((section_name, [t.text for t in gold], [t.text for t in pred]))

In [62]:
f = open('results/macroeconomics-2e-results.txt','w', encoding='utf-8')

for name, gold, pred in terms_for_eval:
    f.write(name + '\n\n')
    i = 0
    j = 0
    while i < len(gold) and j < len(pred):
        if gold[i] in pred:
            if gold[i] == pred[j]:
                s = f'Gold: {gold[i]: <35} Pred: {pred[j]}'
                i += 1
                j +=1
            else:
                s = ' '*42 + f'Pred: {pred[j]}'
                j += 1
        elif i < len(gold):
            s = f'Gold: {gold[i]: <35}'
            i += 1
        else:
            s = ' '*42 + f'Pred: {pred[j]}'
            j += 1
        f.write(s + '\n')
    f.write('\n')
f.close()

# Send terms to csv

In [ ]:
terms_for_eval

In [12]:
import pandas as pd
pd.DataFrame(terms_for_eval, columns=['Section', 'Gold', 'Pred']).to_csv('results/macroeconomics-2e-results.csv', index=False)